# Davis Franklin

## Research question/interests

**I would like to further investigate Apple's market capitalization by specifically looking at the market value during times of innovation of products and software. I am interested in tech and specifically, apple products, and would like to see what innovations worked well and what innovations did not work well for Apple.** I am going to answer this research question by analyzing the slopes of Apple's stock prices in the dataset and compare this information with the release of new Apple products and new updates to Apple's operating systems. I would like the final outcome to be a graph where the user can view individual points on the graph and see product innovations during that time period.

In [1]:
import pandas as pd
pd.read_csv("../data/raw/AAPL.csv")

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.515213,493729600
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.526476,601904800
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.422664,552160000
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.410790,477131200
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.453412,447610800
...,...,...,...,...,...,...,...
3266,2022-12-22,134.350006,134.559998,130.300003,132.229996,132.229996,77852100
3267,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.860001,63814900
3268,2022-12-27,131.380005,131.410004,128.720001,130.029999,130.029999,69007800
3269,2022-12-28,129.669998,131.029999,125.870003,126.040001,126.040001,85438400
